## LIBRARIES AND LOADING DATA

In [1]:
import pandas as pd
import numpy as np
import copy
import json
from numpy import mean
from numpy import std
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.svm import SVC
import nltk

In [2]:
intersection_df = pd.read_json('intersection_df.json',  orient="records", lines = True)

In [3]:
# create dataset
X = intersection_df['text']
y = intersection_df['binary']

## BINARY SVM WITH NESTED STRATIFIED KFOLD CROSS-VALIDATION

In [4]:
#source: https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/?fbclid=IwAR0H80UxAb7r6cVgD2c5kAfqthfYBWOupzLwRk1Mg7l8BDcR9_EcASHHCsc
#source: https://www.geeksforgeeks.org/stratified-k-fold-cross-validation/

pipeline = Pipeline([
                  ('vect', CountVectorizer(max_features=500,min_df=3)), 
                  ('tfidf', TfidfTransformer()), 
                  ('svc', SVC(random_state = 1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X,y):
  # split data
  X_train, X_test = X[train_ix], X[test_ix]
  y_train, y_test = y[train_ix], y[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  #space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  #space['vect__min_df'] = [1,3,5,10]
  space['svc__C'] = [0.1, 1, 10, 100, 1000]
  space['svc__gamma'] = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]
  space['svc__kernel'] = ['linear', 'rbf']
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.713, est=0.677, cfg={'svc__C': 10, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.76      0.87      0.81        15
           1       0.60      0.43      0.50         7

    accuracy                           0.73        22
   macro avg       0.68      0.65      0.66        22
weighted avg       0.71      0.73      0.71        22

>f1score=0.689, est=0.700, cfg={'svc__C': 10, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.74      0.93      0.82        15
           1       0.67      0.29      0.40         7

    accuracy                           0.73        22
   macro avg       0.70      0.61      0.61        22
weighted avg       0.71      0.73      0.69        22

>f1score=0.545, est=0.707, cfg={'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.67     

## MULTI-CLASS CLASSIFICATION

### Catastrophizing vs Other distortion vs Labeling

In [5]:
intersection_df['label_new'].value_counts()

Not Distorted       152
Catastrophizing      43
Other distortion     12
Labeling              9
Name: label_new, dtype: int64

In [6]:
df_with_distortions = intersection_df[intersection_df['binary'] == 1]
df_with_distortions = df_with_distortions.reset_index(drop=True)
df_with_distortions['label_new'].value_counts()

Catastrophizing     43
Other distortion    12
Labeling             9
Name: label_new, dtype: int64

In [7]:
X_multi = df_with_distortions['text']
y_multi = df_with_distortions['label_new']

In [8]:
pipeline = Pipeline([
                  ('vect', CountVectorizer()), 
                  ('tfidf', TfidfTransformer()), 
                  ('svc', SVC(random_state = 1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=9, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X_multi,y_multi):
  # split data
  X_train, X_test = X_multi[train_ix], X_multi[test_ix]
  y_train, y_test = y_multi[train_ix], y_multi[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  space['vect__min_df'] = [1,3,5,10]
  space['svc__C'] = [0.1, 1, 10, 100, 1000]
  space['svc__gamma'] = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]
  space['svc__kernel'] = ['linear', 'rbf']
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.621, est=0.597, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 5000, 'vect__min_df': 10}
                  precision    recall  f1-score   support

 Catastrophizing       0.67      0.80      0.73         5
        Labeling       0.00      0.00      0.00         1
Other distortion       1.00      0.50      0.67         2

        accuracy                           0.62         8
       macro avg       0.56      0.43      0.46         8
    weighted avg       0.67      0.62      0.62         8

>f1score=0.429, est=0.590, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 5000, 'vect__min_df': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.60      0.60      0.60         5
        Labeling       0.00      0.00      0.00         1
Other distortion       0.00      0.00      0.00         1

        accuracy                           0.43         7
       macro avg      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.792, est=0.600, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 5000, 'vect__min_df': 10}
                  precision    recall  f1-score   support

 Catastrophizing       0.83      1.00      0.91         5
        Labeling       0.00      0.00      0.00         1
Other distortion       1.00      1.00      1.00         1

        accuracy                           0.86         7
       macro avg       0.61      0.67      0.64         7
    weighted avg       0.74      0.86      0.79         7



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.571, est=0.603, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 5000, 'vect__min_df': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.80      0.80      0.80         5
        Labeling       0.00      0.00      0.00         1
Other distortion       0.00      0.00      0.00         1

        accuracy                           0.57         7
       macro avg       0.27      0.27      0.27         7
    weighted avg       0.57      0.57      0.57         7



...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.648, est=0.687, cfg={'svc__C': 100, 'svc__gamma': 0.1, 'svc__kernel': 'rbf', 'vect__max_features': 5000, 'vect__min_df': 10}
                  precision    recall  f1-score   support

 Catastrophizing       0.67      1.00      0.80         4
        Labeling       0.00      0.00      0.00         1
Other distortion       1.00      0.50      0.67         2

        accuracy                           0.71         7
       macro avg       0.56      0.50      0.49         7
    weighted avg       0.67      0.71      0.65         7



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.416, est=0.620, cfg={'svc__C': 100, 'svc__gamma': 0.1, 'svc__kernel': 'rbf', 'vect__max_features': 5000, 'vect__min_df': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.57      1.00      0.73         4
        Labeling       0.00      0.00      0.00         1
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.57         7
       macro avg       0.19      0.33      0.24         7
    weighted avg       0.33      0.57      0.42         7



F1-score weighted Mean and Std: 0.597 (0.110)
Max F1-score weighted: 0.7922077922077921


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Catastrophizing vs Other vs Not distorted

In [9]:
intersection_df['label_new_2'].value_counts()

Not Distorted       152
Catastrophizing      43
Other distortion     21
Name: label_new_2, dtype: int64

In [10]:
X_multi = intersection_df['text']
y_multi = intersection_df['label_new_2']

In [11]:
pipeline = Pipeline([
                  ('vect', CountVectorizer()), 
                  ('tfidf', TfidfTransformer()), 
                  ('svc', SVC(random_state = 1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X_multi,y_multi):
  # split data
  X_train, X_test = X_multi[train_ix], X_multi[test_ix]
  y_train, y_test = y_multi[train_ix], y_multi[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  space['vect__min_df'] = [1,3,5,10]
  space['svc__C'] = [0.1, 1, 10, 100, 1000]
  space['svc__gamma'] = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]
  space['svc__kernel'] = ['linear', 'rbf']
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.553, est=0.638, cfg={'svc__C': 10, 'svc__gamma': 0.1, 'svc__kernel': 'rbf', 'vect__max_features': 2000, 'vect__min_df': 1}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         5
   Not Distorted       0.68      1.00      0.81        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.68        22
       macro avg       0.23      0.33      0.27        22
    weighted avg       0.46      0.68      0.55        22



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.506, est=0.680, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 1000, 'vect__min_df': 1}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         5
   Not Distorted       0.65      0.87      0.74        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.59        22
       macro avg       0.22      0.29      0.25        22
    weighted avg       0.44      0.59      0.51        22



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.610, est=0.645, cfg={'svc__C': 100, 'svc__gamma': 0.1, 'svc__kernel': 'rbf', 'vect__max_features': 500, 'vect__min_df': 3}
                  precision    recall  f1-score   support

 Catastrophizing       0.50      0.20      0.29         5
   Not Distorted       0.70      0.93      0.80        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.68        22
       macro avg       0.40      0.38      0.36        22
    weighted avg       0.59      0.68      0.61        22



...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.614, est=0.679, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 5000, 'vect__min_df': 3}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         4
   Not Distorted       0.70      0.93      0.80        15
Other distortion       1.00      0.33      0.50         3

        accuracy                           0.68        22
       macro avg       0.57      0.42      0.43        22
    weighted avg       0.61      0.68      0.61        22

>f1score=0.594, est=0.624, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 5000, 'vect__min_df': 10}
                  precision    recall  f1-score   support

 Catastrophizing       0.17      0.25      0.20         4
   Not Distorted       0.71      0.67      0.69        15
Other distortion       1.00      0.50      0.67         2

        accuracy                           0.57        21
       macro avg      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.588, est=0.638, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 2000, 'vect__min_df': 1}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         4
   Not Distorted       0.74      0.93      0.82        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.67        21
       macro avg       0.25      0.31      0.27        21
    weighted avg       0.53      0.67      0.59        21



F1-score weighted Mean and Std: 0.617 (0.073)
Max F1-score weighted: 0.7688311688311689


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## BINARY - Catastrophizing vs Other

In [12]:
X_multi = df_with_distortions['text']
y_multi = df_with_distortions['label_new_2']

In [13]:
y_multi.value_counts()

Catastrophizing     43
Other distortion    21
Name: label_new_2, dtype: int64

In [14]:
pipeline = Pipeline([
                  ('vect', CountVectorizer()), 
                  ('tfidf', TfidfTransformer()), 
                  ('svc', SVC(random_state = 1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X_multi,y_multi):
  # split data
  X_train, X_test = X_multi[train_ix], X_multi[test_ix]
  y_train, y_test = y_multi[train_ix], y_multi[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  space['vect__min_df'] = [1,3,5,10]
  space['svc__C'] = [0.1, 1, 10, 100, 1000]
  space['svc__gamma'] = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]
  space['svc__kernel'] = ['linear', 'rbf']
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.429, est=0.689, cfg={'svc__C': 100, 'svc__gamma': 0.1, 'svc__kernel': 'rbf', 'vect__max_features': 5000, 'vect__min_df': 10}
                  precision    recall  f1-score   support

 Catastrophizing       0.60      0.60      0.60         5
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.43         7
       macro avg       0.30      0.30      0.30         7
    weighted avg       0.43      0.43      0.43         7

>f1score=0.519, est=0.735, cfg={'svc__C': 1000, 'svc__gamma': 0.1, 'svc__kernel': 'rbf', 'vect__max_features': 5, 'vect__min_df': 1}
                  precision    recall  f1-score   support

 Catastrophizing       0.67      0.80      0.73         5
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.57         7
       macro avg       0.33      0.40      0.36         7
    weighted avg       0.48      0.57      0.52         7

>f1score=1.000, est=0.709, cf

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.667, est=0.709, cfg={'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf', 'vect__max_features': 5000, 'vect__min_df': 10}
                  precision    recall  f1-score   support

 Catastrophizing       0.75      0.75      0.75         4
Other distortion       0.50      0.50      0.50         2

        accuracy                           0.67         6
       macro avg       0.62      0.62      0.62         6
    weighted avg       0.67      0.67      0.67         6

>f1score=1.000, est=0.704, cfg={'svc__C': 10, 'svc__gamma': 1000, 'svc__kernel': 'linear', 'vect__max_features': 5000, 'vect__min_df': 10}
                  precision    recall  f1-score   support

 Catastrophizing       1.00      1.00      1.00         4
Other distortion       1.00      1.00      1.00         2

        accuracy                           1.00         6
       macro avg       1.00      1.00      1.00         6
    weighted avg       1.00      1.00      1.00         6

>f1score=0.815, est=0.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
